# Benchmark of various outlier detection model thresholders

### The models are evaluated by ROC, Precision @ n and execution time on 17 benchmark datasets. All datasets are split (60% for training and 40% for testing). The full result by averaging 10 indepent trials can be found [here]

The thresholders covered in this example include:

1. **IQR: Inter-Quartile Region** 
2. **MAD: Median Absolute Deviation**
3. **FWFM: Full Width at Full Minimum**
4. **YJ: Yeo-Johnson Transformation**
5. **ZSCORE: Z Score**
6. **AUCP: Area Under the Curve Percentage**
7. **QMCD: Quasi-Monte Carlo Discreperancy**
8. **FGD: Fixed Gradient Descent**
9. **DSN: Distance Shift from Normal**
10. **CLF: Trained Classifier**
11. **FILTER: Filtering Based**
12. **WIND: Topological Winding Number**
13. **EB: Elliptical Boundary**
14. **REGR: Regression Intercept**
15. **BOOT: Bootstrap Method**
16. **MCST: Monte Carlo Statistical Tests**
17. **HIST: Histogram Based Methods**
18. **MOLL: Mollifier**
19. **CHAU: Chauvenet's Criterion**
20. **GESD: Generalized Extreme Studentized Deviate**
21. **MTT: Modified Thompson Tau Test**
22. **KARCH: Karcher Mean**
23. **OCSVM: One-Class SVM**
24. **CLUST: Clustering**
25. **DECOMP: Decomposition**
26. **META: Meta-model**
27. **VAE: Variational Autoencoder**
28. **CPD: Change Point Detection**

In [1]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.sampling import Sampling
from pyod.models.kde import KDE
from pyod.models.knn import KNN
from pyod.models.mcd import MCD
from pyod.models.iforest import IForest

from pythresh.thresholds.iqr import IQR
from pythresh.thresholds.mad import MAD
from pythresh.thresholds.fwfm import FWFM
from pythresh.thresholds.yj import YJ
from pythresh.thresholds.zscore import ZSCORE
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.qmcd import QMCD
from pythresh.thresholds.fgd import FGD
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.clf import CLF
from pythresh.thresholds.filter import FILTER
from pythresh.thresholds.wind import WIND
from pythresh.thresholds.eb import EB
from pythresh.thresholds.regr import REGR
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.mcst import MCST
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.moll import MOLL
from pythresh.thresholds.chau import CHAU
from pythresh.thresholds.gesd import GESD
from pythresh.thresholds.mtt import MTT
from pythresh.thresholds.karch import KARCH
from pythresh.thresholds.ocsvm import OCSVM
from pythresh.thresholds.clust import CLUST
from pythresh.thresholds.decomp import DECOMP
from pythresh.thresholds.meta import META
from pythresh.thresholds.vae import VAE
from pythresh.thresholds.cpd import CPD

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

INFO: Using numpy backend


In [2]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc', 'IQR', 'MAD', 'FWFM', 
              'YJ', 'ZSCORE', 'AUCP', 'QMCD', 'FGD', 'DSN', 'CLF', 'FILTER', 'WIND', 
              'EB', 'REGR', 'BOOT', 'MCST', 'HIST', 'MOLL', 'CHAU', 'GESD', 'MTT', 'KARCH',
              'OCSVM', 'CLUST', 'DECOMP', 'META', 'VAE', 'CPD']
roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

clf = IForest()


for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    thresholders = {
        'Inter-Quartile Region (IQR)':IQR(),
        'Median Absolute Deviation (MAD)':MAD(),
        'Full Width at Full Minimum (FWFM)':FWFM(),
        'Yeo-Johnson Transformation (YJ)': YJ(),
        'Z Score (ZSCORE)': ZSCORE(),
        'AUC Percentage (AUCP)': AUCP(),
        'Quasi-Monte Carlo Discreperancy (QMCD)': QMCD(),
        'Fixed Gradient Descent (FGD)': FGD(),
        'Distance Shift from Normal (DSN)': DSN(),
        'Trained Classifier (CLF)': CLF(),
        'Filtering Based (FILTER)': FILTER(),
        'Topological Winding Number (WIND)': WIND(),
        'Elliptical Boundary (EB)': EB(),
        'Regression Intercept (REGR)': REGR(),
        'Bootstrap Method (BOOT)': BOOT(),
        'Monte Carlo Statistical Tests (MCST)': MCST(),
        'Histogram Based Methods (HIST)': HIST(),
        'Mollifier (MOLL)': MOLL(),
        "Chauvenet's Criterion (CHAU)": CHAU(),
        'Generalized Extreme Studentized Deviate (GESD)': GESD(),
        'Modified Thompson Tau Test (MTT)': MTT(),
        'Karcher Mean (KARCH)': KARCH(),
        'One-Class SVM (OCSVM)': OCSVM(),
        'Clustering (CLUST)': CLUST(),
        'Decomposition (DECOMP)': DECOMP(),
        'Meta-model (META)': META(),
        'Variational Autoencoder (VAE)': VAE(),
        'Change Point Detection (CPD)': CPD()
    }
    
    clf.fit(X_train_norm)
    scores = clf.decision_scores_
    
    for thres_name, thres in thresholders.items():
        t0 = time()
        pred = thres.eval(scores)
        contam = np.sum(pred)/len(pred)
        
        if contam<=0: contam=1e-3
        if contam>0.5: contam=0.5
        
        clf = IForest(contamination=contam)
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...

... Processing cardio.mat ...

... Processing glass.mat ...

... Processing ionosphere.mat ...

... Processing letter.mat ...

... Processing lympho.mat ...

... Processing mnist.mat ...

... Processing musk.mat ...

... Processing optdigits.mat ...

... Processing pendigits.mat ...

... Processing pima.mat ...

... Processing satellite.mat ...

... Processing satimage-2.mat ...

... Processing vertebral.mat ...

... Processing vowels.mat ...

... Processing wbc.mat ...


In [3]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META,VAE,CPD
0,arrhythmia,452,274,14.6018,0.6802,0.7033,0.7327,1.0153,0.8093,0.9667,...,0.7568,0.9455,0.6973,0.6678,0.6807,0.9181,0.7,5.2042,3.6113,0.7462
0,cardio,1831,21,9.6122,0.8769,0.6997,0.755,1.4131,0.6646,0.8548,...,0.6581,1.012,0.6876,0.8208,0.6851,1.0136,0.9945,5.5907,12.2794,1.1289
0,glass,214,9,4.2056,0.4692,0.479,0.4592,0.6713,0.457,0.4714,...,0.4536,0.543,0.4913,0.4507,0.5107,0.6071,0.518,5.0809,1.7559,0.4886
0,ionosphere,351,33,35.8974,0.4964,0.5403,0.5017,0.8091,0.4851,0.5201,...,0.4851,0.5446,0.4856,0.4886,0.4864,0.5153,0.5243,4.9266,2.6213,0.5398
0,letter,1600,32,6.25,0.6317,0.6441,0.7708,1.5373,0.6597,0.9568,...,0.6631,0.9297,0.6793,0.6769,0.6362,0.9533,0.8934,5.4959,10.3348,1.0
0,lympho,148,18,4.0541,0.4485,0.448,0.4506,0.7765,0.4474,0.4732,...,0.4579,0.473,0.4486,0.451,0.4531,0.4719,0.4626,4.818,1.522,0.4715
0,mnist,7603,100,9.2069,1.9818,1.9912,4.0679,5.6447,1.9882,4.1054,...,1.9993,3.3711,2.0136,2.7804,2.004,10.7586,5.5869,7.1832,52.801,6.2512
0,musk,3062,166,3.1679,1.441,1.3975,1.7386,2.9593,1.3969,1.8441,...,1.3759,1.9186,1.3876,1.5183,1.3794,2.2387,2.0764,6.0576,19.8868,2.3047
0,optdigits,5216,64,2.8758,1.3574,1.3125,2.3117,3.5957,1.3156,2.3899,...,1.3174,2.2497,1.3371,1.729,1.3763,4.0446,3.0873,6.2352,32.1622,3.4884
0,pendigits,6870,16,2.2707,1.1366,1.0943,2.8716,3.5842,1.09,2.8095,...,1.0843,2.4225,1.1296,1.7533,1.0915,7.7991,4.2287,7.589,47.5405,4.8924


In [4]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META,VAE,CPD
0,arrhythmia,452,274,14.6018,0.8368,0.8443,0.845,0.8224,0.8329,0.8462,...,0.8497,0.8289,0.8466,0.7967,0.8466,0.8613,0.8371,0.8623,0.856,0.8494
0,cardio,1831,21,9.6122,0.8966,0.9315,0.9129,0.9462,0.9168,0.9063,...,0.9246,0.9187,0.9209,0.9102,0.9112,0.9318,0.9211,0.9426,0.9374,0.9176
0,glass,214,9,4.2056,0.6494,0.642,0.6568,0.6198,0.6173,0.642,...,0.6543,0.6988,0.637,0.6296,0.6444,0.6519,0.6519,0.6568,0.7235,0.6543
0,ionosphere,351,33,35.8974,0.7709,0.7881,0.7888,0.7904,0.7838,0.7945,...,0.7719,0.7634,0.7771,0.7698,0.776,0.7835,0.7831,0.7762,0.7833,0.7851
0,letter,1600,32,6.25,0.5931,0.6259,0.6047,0.5921,0.6124,0.629,...,0.6303,0.6477,0.5834,0.6392,0.6181,0.606,0.6365,0.6116,0.6298,0.6172
0,lympho,148,18,4.0541,1.0,1.0,0.9942,1.0,1.0,0.9883,...,0.9942,0.9942,1.0,1.0,1.0,1.0,1.0,0.9942,1.0,1.0
0,mnist,7603,100,9.2069,0.8253,0.7751,0.7792,0.8097,0.8187,0.7784,...,0.7896,0.7719,0.7975,0.7875,0.786,0.8098,0.8086,0.812,0.8173,0.7683
0,musk,3062,166,3.1679,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.9953,0.9999,1.0,0.9996,1.0,1.0
0,optdigits,5216,64,2.8758,0.6331,0.7157,0.6861,0.6685,0.7041,0.6988,...,0.6418,0.7084,0.7219,0.7241,0.747,0.6786,0.6922,0.7615,0.7216,0.6264
0,pendigits,6870,16,2.2707,0.9332,0.9329,0.9255,0.9358,0.9495,0.9427,...,0.9424,0.9429,0.9313,0.9229,0.923,0.9453,0.9362,0.9424,0.935,0.9578


In [5]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META,VAE,CPD
0,arrhythmia,452,274,14.6018,0.5714,0.5,0.5714,0.5,0.5357,0.5357,...,0.6071,0.5714,0.5357,0.5,0.5357,0.6071,0.4643,0.5714,0.5357,0.5357
0,cardio,1831,21,9.6122,0.4429,0.5,0.4429,0.5857,0.4714,0.4286,...,0.4857,0.5,0.5,0.4714,0.4429,0.5143,0.4286,0.5714,0.5429,0.4714
0,glass,214,9,4.2056,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
0,ionosphere,351,33,35.8974,0.5,0.5435,0.5435,0.5435,0.5435,0.5435,...,0.5217,0.5217,0.587,0.5217,0.5435,0.5652,0.5435,0.5435,0.5435,0.5217
0,letter,1600,32,6.25,0.0732,0.0732,0.0732,0.0732,0.0976,0.0976,...,0.0976,0.0732,0.0976,0.0488,0.0976,0.0976,0.0732,0.0976,0.0976,0.122
0,lympho,148,18,4.0541,1.0,1.0,0.6667,1.0,1.0,0.6667,...,0.6667,0.6667,1.0,1.0,1.0,1.0,1.0,0.6667,1.0,1.0
0,mnist,7603,100,9.2069,0.3111,0.2481,0.2778,0.2815,0.3333,0.2778,...,0.2815,0.2593,0.2741,0.3,0.2889,0.3074,0.3259,0.3037,0.3,0.2593
0,musk,3062,166,3.1679,0.9756,1.0,1.0,1.0,0.9756,1.0,...,1.0,1.0,1.0,1.0,0.7317,0.9756,1.0,0.9512,1.0,1.0
0,optdigits,5216,64,2.8758,0.0154,0.0308,0.0,0.0,0.0,0.0,...,0.0308,0.0615,0.0769,0.0308,0.0462,0.0154,0.0,0.0,0.0308,0.0
0,pendigits,6870,16,2.2707,0.2258,0.3065,0.2419,0.2903,0.2419,0.2419,...,0.3226,0.2903,0.1774,0.2581,0.2097,0.3226,0.2258,0.2581,0.1935,0.3226
